In [1]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [2]:
# 📦 Imports

In [3]:
import pandas as pd
import torch
import random
from datasets import load_dataset
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [4]:
# 🗃️ Load dataset

In [5]:
dataset = load_dataset("amazon_polarity", split="train[:10000]")
df = pd.DataFrame(dataset)
df.rename(columns={"label": "sentiment", "content": "text"}, inplace=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [6]:
# 🔧 Simulate aspects
aspects = ['battery', 'display', 'performance', 'price', 'design']
df["aspect"] = df["text"].apply(lambda x: random.choice(aspects))

In [7]:
# 🔁 Convert to 3-class sentiment
neutral_idx = df.sample(frac=0.2, random_state=42).index
df.loc[neutral_idx, "sentiment"] = 1  # neutral
df.loc[df["sentiment"] == 1, "sentiment"] = 2  # convert original positive to 2

In [8]:
# 🧾 Format: "[aspect]: review"
df["input_text"] = df["aspect"] + ": " + df["text"]
df = df.sample(n=3000, random_state=42).reset_index(drop=True)

In [9]:
# ✂️ Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["input_text"].tolist(),
    df["sentiment"].tolist(),
    test_size=0.2,
    random_state=42
)

In [10]:
# 🔠 Tokenize
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
# 🔗 Create Dataset class
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [12]:
# 🧠 Load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# ⚙️ TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    do_eval=True,  # runs evaluation after training
    logging_steps=10
)

In [14]:
# 📊 Define compute_metrics (accuracy and F1)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

In [15]:
# 🧠 Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [16]:
# 🚀 Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raja-n (raja-n-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.536100
20,0.401600
30,0.248500
40,0.614300
50,0.443600
60,0.589500
70,0.497700
80,0.536500
90,0.446700
100,0.459300


TrainOutput(global_step=900, training_loss=0.45346306959788, metrics={'train_runtime': 198.9201, 'train_samples_per_second': 36.195, 'train_steps_per_second': 4.524, 'total_flos': 473604151910400.0, 'train_loss': 0.45346306959788, 'epoch': 3.0})

In [17]:
# ✅ Final Evaluation
metrics = trainer.evaluate()
print("Final Evaluation Metrics:", metrics)

Final Evaluation Metrics: {'eval_loss': 0.40183624625205994, 'eval_accuracy': 0.855, 'eval_f1': 0.788167115902965, 'eval_runtime': 4.8846, 'eval_samples_per_second': 122.836, 'eval_steps_per_second': 15.354, 'epoch': 3.0}


In [24]:
# 🔍 Predict sentiment for a given review and aspect
def predict_sentiment(aspect, review_text):
    input_text = f"{aspect}: {review_text}"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    # Move inputs to same device as model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        pred_label = torch.argmax(outputs.logits, dim=1).item()

    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_map[pred_label]


In [25]:
# 🧪 Example Predictions
examples = [
    ("battery", "The battery lasts all day and charges quickly."),
    ("display", "The screen resolution is terrible."),
    ("price", "For the price, this product is unbeatable."),
    ("performance", "It's fast and handles everything I throw at it."),
    ("design", "Looks cheap and feels flimsy.")
]

for aspect, review in examples:
    sentiment = predict_sentiment(aspect, review)
    print(f"Aspect: {aspect.ljust(12)} | Sentiment: {sentiment} | Review: {review}")


Aspect: battery      | Sentiment: Positive | Review: The battery lasts all day and charges quickly.
Aspect: display      | Sentiment: Positive | Review: The screen resolution is terrible.
Aspect: price        | Sentiment: Positive | Review: For the price, this product is unbeatable.
Aspect: performance  | Sentiment: Positive | Review: It's fast and handles everything I throw at it.
Aspect: design       | Sentiment: Positive | Review: Looks cheap and feels flimsy.
